In [2]:
import json
import requests
from config import api_key

In [2]:
# Basic structure of API call:
# year.dev-category.dev-friendly-variable-name.

In [28]:
# Params
base_url = 'https://api.data.gov/ed/collegescorecard/v1/schools.json?'
state = 'school.state=FL&fields=id,school.name,school.state' 
year = 2018
cost_per = f'{year}.cost.attendance.academic_year,{year}.cost.attendance.program_year'
predom_type = 'school.degrees_awarded.predominant_recoded'

# Grab the base url
url = f'{base_url}{state},{cost_per},{predom_type}&api_key={api_key}'
response = requests.get(url).json()
response

{'metadata': {'total': 387, 'page': 0, 'per_page': 20},
 'results': [{'school.name': 'Palm Beach Academy of Health & Beauty',
   'school.state': 'FL',
   'id': 449490,
   'school.degrees_awarded.predominant_recoded': 1,
   '2018.cost.attendance.program_year': 25572,
   '2018.cost.attendance.academic_year': None},
  {'school.name': 'SABER College',
   'school.state': 'FL',
   'id': 449506,
   'school.degrees_awarded.predominant_recoded': 2,
   '2018.cost.attendance.program_year': 19233,
   '2018.cost.attendance.academic_year': None},
  {'school.name': 'Pensacola School of Massage Therapy & Health Careers',
   'school.state': 'FL',
   'id': 449533,
   'school.degrees_awarded.predominant_recoded': 1,
   '2018.cost.attendance.program_year': 21608,
   '2018.cost.attendance.academic_year': None},
  {'school.name': 'Paul Mitchell the School-Tampa',
   'school.state': 'FL',
   'id': 449959,
   'school.degrees_awarded.predominant_recoded': 1,
   '2018.cost.attendance.program_year': 21596,
   '2

#specifying areas of studies Key:
    1: Undergraduate Certificates or Diplomas
    2: Associate’s Degrees
    3: Bachelor’s Degrees
    4: Post-Baccalaureate Certificates
    5: Master’s Degrees
    6: Doctoral Degrees
    7: First Professional Degrees
    8: Graduate / Professional Certificates